In [9]:
import os
import openai
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']
from GraceAgent.LLMPlanner import GracePlanner
from Utils.PromptTemplateBuilder import PromptTemplateBuilder
from langchain_core.prompts.chat import SystemMessagePromptTemplate,ChatPromptTemplate

In [10]:
from Tools import *
from Tools.PythonTool import CSVAnalyser

In [11]:
tools = [
    document_generation_tool,
    directory_inspection_tool,
    CSV_inspection_tool,
    finish_placeholder,
    CSVAnalyser(
        prompt_path="./prompts/Tools",
        prompt_file="csv_analyser.json"
    ).as_tool()
]

In [22]:
model = ChatOpenAI(model="gpt-4",temperature=0)
planner = GracePlanner(model, tools, stop=['<END_OF_PLAN>'])

In [13]:
print(planner.plan({'input':"我想知道数据集中的男女比例信息",'previous_steps':""}))

steps=[Step(value='读取数据集文件，获取性别信息和数量。'), Step(value='计算男性数量。'), Step(value='计算女性数量。'), Step(value='计算男女比例。\n<END_OF_PLAN>')]


In [23]:
from GraceAgent import GraceExecutor

executor = GraceExecutor(
        llm=model,
        prompts_path="./prompts/main",
        tools=tools,
        work_dir='datasets',
        main_prompt_file='executor.json',
        final_prompt_file='final_step.json',
        max_thought_steps=10,
    )

In [24]:
executor.run({"previous_steps":"","current_step":"读取数据集以获取男女信息。"})

>>>>Round :0<<<<
关键概念: 数据集，男女信息
概念拆解: 
- 数据集
 - 数据集的路径
 - 数据集的内容
- 男女信息
 - 数据集中男女信息的列名
 - 数据集中男女信息的具体数据

反思:
 - 目前我还没有获取到任何关键概念的取值。
 - 我需要先获取数据集的路径和内容，然后才能获取男女信息的列名和具体数据。

思考: 
 - 我需要先获取数据集的路径，然后才能读取数据集的内容。
 - 读取数据集的内容后，我可以获取男女信息的列名和具体数据。
 - 我可以通过ListDirectory工具来获取数据集的路径。

推理: 
 - 根据我的反思和思考，我决定先获取数据集的路径。
 - 我可以通过使用ListDirectory工具来获取数据集的路径。

计划: 
 - 我计划使用ListDirectory工具来获取数据集的路径。
 - 我将使用"datasets"作为ListDirectory工具的参数，因为这是给定的数据集的相对路径。

执行动作/工具：
```
{
"name": "ListDirectory",
"args": {"path": "datasets"}
}
```
----
结果raw_target_data.csv
raw_events_data.csv
raw_labtest_data.csv
>>>>Round :1<<<<
关键概念: 
- 数据集
- 男女信息

概念拆解: 
- 数据集
 - 数据集的路径: datasets/raw_target_data.csv, datasets/raw_events_data.csv, datasets/raw_labtest_data.csv
 - 数据集的内容
- 男女信息
 - 数据集中男女信息的列名
 - 数据集中男女信息的具体数据

反思:
 - 我已经获取到了数据集的路径，包括raw_target_data.csv, raw_events_data.csv, raw_labtest_data.csv。
 - 我还没有获取到数据集的内容，以及男女信息的列名和具体数据。

思考: 
 - 我需要先读取数据集的内容，然后才能获取男女信息的列名和具体数据。
 - 我可以通过InspectCSV工具来读取数据集的内容。

推理: 
 - 根据我的反思和思考，我决定先读

Python REPL can execute arbitrary code. Use with caution.



----
结果The male to female ratio is: 1.4834437086092715

>>>>Round :4<<<<
关键概念: 
- 数据集
 - 数据集的路径: datasets/raw_target_data.csv, datasets/raw_events_data.csv, datasets/raw_labtest_data.csv
 - 数据集的内容: raw_target_data.csv的部分内容已获取，raw_events_data.csv的部分内容已获取
- 男女信息
 - 数据集中男女信息的列名: Sex
 - 数据集中男女信息的具体数据: 男女比例为1.4834437086092715

概念拆解: 
- 数据集
 - 数据集的路径: datasets/raw_target_data.csv, datasets/raw_events_data.csv, datasets/raw_labtest_data.csv
 - 数据集的内容: raw_target_data.csv的部分内容已获取，raw_events_data.csv的部分内容已获取，但未读取raw_labtest_data.csv的内容
- 男女信息
 - 数据集中男女信息的列名: Sex
 - 数据集中男女信息的具体数据: 男女比例为1.4834437086092715

反思:
 - 我已经获取到了数据集的路径，包括raw_target_data.csv, raw_events_data.csv, raw_labtest_data.csv。
 - 我已经读取了raw_target_data.csv和raw_events_data.csv的部分内容，发现了男女信息的列名为Sex。
 - 我已经获取到了男女信息的具体数据，男女比例为1.4834437086092715。

思考: 
 - 我已经完成了当前的任务，获取到了男女信息的具体数据。

推理: 
 - 根据我的反思和思考，我决定结束当前的任务。

计划: 
 - 我计划使用FINISH工具来结束当前的任务。

执行动作/工具：
```
{
"name": "FINISH"
}
```
----
FINISH


'我已经成功读取了数据集并获取了男女信息。在数据集中，男女信息的列名为"Sex"。通过分析，我得到了男女比例，男性与女性的比例约为1.48:1。'

In [30]:
prompt = PromptTemplateBuilder(prompt_path="./prompts/executor", prompt_file="executor.json").build(
    tools=tools
).partial(
            work_dir='./datasets',
    current_step="",
format_instructions = "", 
input="", 
previous_step="", 
short_term_memory=""
        )

In [33]:
print(prompt.format())

你是强大的 AI 助手的具体执行模块,面对着用户提出的问题，planner 已将将其进行分解成一个个具体的小任务，你只是需要使用工具与指令自动化解决当前面临的具体任务！！！
你必须遵循以下约束来完成任务。
1. 每次你的决策只使用一种工具，你可以使用任意多次。
2. 确保你调用的指令或使用的工具在下述给定的工具列表中。
3. 确保你的回答不会包含违法或有侵犯性的信息。
4. 你不需要保证用户提出的问题是否解决，你的责任只是完成当前面临的具体任务，如果你已经完成当前面临的具体任务，就确保使用"FINISH"指令结束，不要多管闲事。
5. 请详细分析当前的任务执行记录，其中包含了你之前执行的记录和内容，你必须根据这些来进行后续思考判定。
5. 用中文思考和输出。
6. 请务必注意！！！如果执行某个指令或工具失败，尝试改变参数或参数格式再次调用。
7. 你生成的回复必须遵循上文中给定的事实信息。不可以编造信息。 DO NOT MAEK UP INFORMATION.
8. 如果得到的结果不正确，尝试更换表达方式。
9. 已经得到的信息，不要反复查询。
10. 确保你生成的动作是可以精确执行的。动作中可以包括具体方法和目标输出。
11. 看到一个概念时尝试获取它的精确定义，并分析从哪些输入可以得到它的具体取值。
12. 生成一个自然语言查询时，请在查询中包含全部的已知信息。
13. 在执行分析或计算动作前，确保分析或计算中涉及的所有的子概念都已经得到了定义。
14. 你不可以打印一个文件的全部内容，这样的操作代价太大，且会造成不可预期的后果，是被严格禁止的，

你已经解决的任务是：

你当前面临的具体任务是：

如果此任务显示“无”、“没有了”、“已完成”或者类似表达，你直接输出下述工具中的 FINISH 即可。

你需要的所有文件资料都在以下目录，在需要指定相关路径的时候一定优先考虑该路径：
dir_path=./datasets
注意这是相对路径

当前的任务执行记录：


你可以使用以下工具或指令，它们又被称为动作或actions：
1. GenerateDocument:GenerateDocument(query: str) - 根据需求描述生成一篇正式文档,                    args json schema:{"query": {"titl